<a href="https://colab.research.google.com/github/ThanuMahee12/ayush-vision/blob/ssd/SSDRoot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="KvYzBayallmQzuRDfNc3")
project = rf.workspace("ayushvision").project("ayushvision_root")
version = project.version(2)
dataset = version.download("coco")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to ayushvision_root-2 in coco:: 100%|██████████| 488/488 [00:00<00:00, 5402.35it/s]


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install roboflow torch torchvision


In [4]:
train_dir = f"/content/ayushvision_root-2/train"
valid_dir = f"/content/ayushvision_root-2/valid"

In [35]:
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import CocoDetection
import torchvision.transforms as T

transform =T.Compose([
    T.Resize((300, 300)),  # Resize to expected input size
    T.ToTensor(),
])

# Load COCO format dataset
train_dataset = CocoDetection(root=train_dir, annFile=f"{train_dir}/_annotations.coco.json", transform=transform)
valid_dataset = CocoDetection(root=valid_dir, annFile=f"{valid_dir}/_annotations.coco.json", transform=transform)

# DataLoader to load data in batches
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=False, num_workers=4)


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [36]:
# Load the pre-trained SSD model and modify it for your custom dataset
weights = SSDLite320_MobileNet_V3_Large_Weights.DEFAULT
model = ssdlite320_mobilenet_v3_large(weights=weights)

# Define number of classes (including background)
num_classes = len(train_dataset.coco.getCatIds()) + 1  # Number of classes + background

# Modify the classification head to match the number of classes
# Access the classifier directly and replace it with a new one
# Get the input channels from the first layer of the classification head
# Check the structure of model.head.classification_head for Roboflow models
# It might be different from the torchvision implementation
print(model.head.classification_head)

# Instead of accessing conv[0], try accessing the first layer directly
# Check the model's structure to confirm the correct layer name


SSDLiteClassificationHead(
  (module_list): ModuleList(
    (0): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(672, 672, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=672, bias=False)
        (1): BatchNorm2d(672, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): Conv2d(672, 546, kernel_size=(1, 1), stride=(1, 1))
    )
    (1): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(480, 480, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=480, bias=False)
        (1): BatchNorm2d(480, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): Conv2d(480, 546, kernel_size=(1, 1), stride=(1, 1))
    )
    (2): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=512, bias=False)
        (1): BatchNorm2d(512, eps=0.001, momentum=0.03, a

In [41]:
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import CocoDetection
from torchvision import transforms as T
from torchvision.models.detection import ssdlite320_mobilenet_v3_large, SSDLite320_MobileNet_V3_Large_Weights

# Define transformations
transform =T.Compose([
    T.Resize((300, 300)),  # Resize to expected input size
    T.ToTensor(),
])

def collate_fn(batch):
    images, targets = zip(*batch)  # Unzip the batch into images and targets
    return list(images), list(targets)

# Load COCO dataset in Roboflow directory structure
train_dataset = CocoDetection(root=train_dir, annFile=f"{train_dir}/_annotations.coco.json", transform=transform)
valid_dataset = CocoDetection(root=valid_dir, annFile=f"{valid_dir}/_annotations.coco.json", transform=transform)

# Use DataLoader with the custom collate function
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4, collate_fn=collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=False, num_workers=4, collate_fn=collate_fn)
# Define number of classes (including background)
num_classes = len(train_dataset.coco.getCatIds()) + 1  # Number of classes + background
# Load the pre-trained SSD model and modify it for your custom dataset
weights = SSDLite320_MobileNet_V3_Large_Weights.DEFAULT
model = ssdlite320_mobilenet_v3_large(weights=weights)



# Get the input channels from the first layer


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [42]:
for i in range(len(model.head.classification_head.module_list)):
    conv_layer = model.head.classification_head.module_list[i][1]  # Get the Conv2d layer
    model.head.classification_head.module_list[i][1] = torch.nn.Conv2d(
        in_channels=conv_layer.in_channels,
        out_channels=num_classes * 4,  # 4 anchors per bounding box
        kernel_size=conv_layer.kernel_size,
        stride=conv_layer.stride,
        padding=conv_layer.padding
    )

In [43]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [44]:
num_epochs = 10

In [46]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, targets in train_loader:
        images = [img.to(device) for img in images]

        new_targets = []
        for target_list in targets:  # targets is a list of lists
            for t in target_list:  # target_list is a list of dictionaries
                new_target = {}

                # Extract boxes and labels
                boxes = torch.tensor(t['bbox']).reshape(-1, 4)  # Assuming 'bbox' is a list of [x_min, y_min, x_max, y_max]

                # Validate boxes: Keep only boxes with positive width and height
                valid_boxes = []
                for box in boxes:
                    x_min, y_min, x_max, y_max = box
                    if x_max > x_min and y_max > y_min:  # Valid box condition
                        valid_boxes.append(box)

                if valid_boxes:  # If there are valid boxes
                    new_target['boxes'] = torch.stack(valid_boxes).to(device)  # Move valid boxes to device
                else:
                    continue  # Skip if no valid boxes

                # Extract labels (make sure it corresponds to valid boxes)
                labels = torch.tensor([t['category_id']] * len(valid_boxes))  # Assuming category_id applies to all valid boxes
                new_target['labels'] = labels.to(device)  # Move labels to device

                new_targets.append(new_target)

        if not new_targets:  # If no valid targets, skip this iteration
            continue

        optimizer.zero_grad()
        loss_dict = model(images, new_targets)
        total_loss = sum(loss for loss in loss_dict.values())

        total_loss.backward()
        optimizer.step()

        running_loss += total_loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader)}")


RuntimeError: shape '[8, -1, 91, 20, 20]' is invalid for input of size 38400

In [47]:
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import CocoDetection
from torchvision import transforms as T
from torchvision.models.detection import ssdlite320_mobilenet_v3_large, SSDLite320_MobileNet_V3_Large_Weights

# Define transformations
transform =T.Compose([
    T.Resize((300, 300)),  # Resize to expected input size
    T.ToTensor(),
])

def collate_fn(batch):
    images, targets = zip(*batch)  # Unzip the batch into images and targets
    return list(images), list(targets)

# Load COCO dataset in Roboflow directory structure
train_dataset = CocoDetection(root=train_dir, annFile=f"{train_dir}/_annotations.coco.json", transform=transform)
valid_dataset = CocoDetection(root=valid_dir, annFile=f"{valid_dir}/_annotations.coco.json", transform=transform)

# Use DataLoader with the custom collate function
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4, collate_fn=collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=False, num_workers=4, collate_fn=collate_fn)
# Define number of classes (including background)
num_classes = len(train_dataset.coco.getCatIds()) + 1  # Number of classes + background
# Load the pre-trained SSD model and modify it for your custom dataset
weights = SSDLite320_MobileNet_V3_Large_Weights.DEFAULT
model = ssdlite320_mobilenet_v3_large(weights=weights)
for i in range(len(model.head.classification_head.module_list)):
    conv_layer = model.head.classification_head.module_list[i][1]  # Get the Conv2d layer
    model.head.classification_head.module_list[i][1] = torch.nn.Conv2d(
        in_channels=conv_layer.in_channels,
        out_channels=num_classes * 4,  # 4 anchors per bounding box
        kernel_size=conv_layer.kernel_size,
        stride=conv_layer.stride,
        padding=conv_layer.padding
    )
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, targets in train_loader:
        images = [img.to(device) for img in images]

        new_targets = []
        for target_list in targets:  # targets is a list of lists
            for t in target_list:  # target_list is a list of dictionaries
                new_target = {}

                # Extract boxes and labels
                boxes = torch.tensor(t['bbox']).reshape(-1, 4)  # Assuming 'bbox' is a list of [x_min, y_min, x_max, y_max]

                # Validate boxes: Keep only boxes with positive width and height
                valid_boxes = []
                for box in boxes:
                    x_min, y_min, x_max, y_max = box
                    if x_max > x_min and y_max > y_min:  # Valid box condition
                        valid_boxes.append(box)

                if valid_boxes:  # If there are valid boxes
                    new_target['boxes'] = torch.stack(valid_boxes).to(device)  # Move valid boxes to device
                else:
                    continue  # Skip if no valid boxes

                # Extract labels (make sure it corresponds to valid boxes)
                labels = torch.tensor([t['category_id']] * len(valid_boxes))  # Assuming category_id applies to all valid boxes
                new_target['labels'] = labels.to(device)  # Move labels to device

                new_targets.append(new_target)

        if not new_targets:  # If no valid targets, skip this iteration
            continue

        optimizer.zero_grad()
        loss_dict = model(images, new_targets)
        total_loss = sum(loss for loss in loss_dict.values())

        total_loss.backward()
        optimizer.step()

        running_loss += total_loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader)}")


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [77]:
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import CocoDetection
from torchvision import transforms as T
from torchvision.models.detection import ssdlite320_mobilenet_v3_large, SSDLite320_MobileNet_V3_Large_Weights
import numpy as np

# Define transformations
transform = T.Compose([
    T.Resize((300, 300)),  # Resize to expected input size
    T.ToTensor(),
])

def collate_fn(batch):
    images, targets = zip(*batch)  # Unzip the batch into images and targets
    return list(images), list(targets)

# Load COCO dataset in Roboflow directory structure
train_dir = "/content/ayushvision_root-2/train"
valid_dir = "/content/ayushvision_root-2/valid"
train_dataset = CocoDetection(root=train_dir, annFile=f"{train_dir}/_annotations.coco.json", transform=transform)
valid_dataset = CocoDetection(root=valid_dir, annFile=f"{valid_dir}/_annotations.coco.json", transform=transform)

# Use DataLoader with the custom collate function
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4, collate_fn=collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=False, num_workers=4, collate_fn=collate_fn)

# Define number of classes (1 for the plant + 1 for background)
num_classes = 2  # 1 for plant + 1 for background

# Load the pre-trained SSD model and modify it for your custom dataset
weights = SSDLite320_MobileNet_V3_Large_Weights.DEFAULT
model = ssdlite320_mobilenet_v3_large(weights=weights)

# Modify the model's classification head for the new number of classes
model.head.classification_head.num_classes = num_classes  # Set to the number of classes

# Move the model to GPU or CPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Function to calculate IoU
def calculate_iou(box, boxes):
    box = box.reshape(1, -1)  # Reshape to (1, 4) for calculations
    x1 = np.maximum(box[:, 0], boxes[:, 0])
    y1 = np.maximum(box[:, 1], boxes[:, 1])
    x2 = np.minimum(box[:, 2], boxes[:, 2])
    y2 = np.minimum(box[:, 3], boxes[:, 3])

    inter_area = np.maximum(0, x2 - x1) * np.maximum(0, y2 - y1)
    box_area = (box[0][2] - box[0][0]) * (box[0][3] - box[0][1])
    boxes_area = (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1])

    union_area = box_area + boxes_area - inter_area
    iou = inter_area / union_area
    iou[np.isnan(iou)] = 0  # Handle division by zero

    return iou

# Function to calculate accuracy
def calculate_accuracy(predictions, targets, threshold=0.5):
    correct_predictions = 0
    total_predictions = 0

    for target, prediction in zip(targets, predictions):
        if prediction is not None and len(prediction['boxes']) > 0:
            pred_boxes = prediction['boxes'].cpu().numpy()
            pred_labels = prediction['labels'].cpu().numpy()

            # Gather true boxes for the current target
            true_boxes = torch.tensor([t['bbox'] for t in target]).reshape(-1, 4).cpu().numpy()

            for pred_box in pred_boxes:
                # Calculate IoU with true boxes
                iou = calculate_iou(pred_box, true_boxes)
                # Check if there is at least one IoU that exceeds the threshold
                if np.any(iou >= threshold):
                    correct_predictions += 1
            total_predictions += len(pred_boxes)

    return correct_predictions / total_predictions if total_predictions > 0 else 0

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, targets in train_loader:
        images = [img.to(device) for img in images]

        new_targets = []
        for target_list in targets:  # targets is a list of lists
            for t in target_list:  # target_list is a list of dictionaries
                new_target = {}

                # Extract boxes and labels
                boxes = torch.tensor(t['bbox']).reshape(-1, 4)  # Assuming 'bbox' is a list of [x_min, y_min, width, height]

                # Validate boxes: Keep only boxes with positive width and height
                valid_boxes = []
                for box in boxes:
                    x_min, y_min, width, height = box
                    x_max = x_min + width
                    y_max = y_min + height
                    if x_max > x_min and y_max > y_min:  # Valid box condition
                        valid_boxes.append([x_min, y_min, x_max, y_max])

                if valid_boxes:  # If there are valid boxes
                    new_target['boxes'] = torch.tensor(valid_boxes).to(device)  # Move valid boxes to device
                    new_target['labels'] = torch.tensor([1] * len(valid_boxes)).to(device)  # Class ID of the detected plant
                    new_targets.append(new_target)

        if not new_targets:  # If no valid targets, skip this iteration
            continue

        optimizer.zero_grad()
        loss_dict = model(images, new_targets)
        total_loss = sum(loss for loss in loss_dict.values())

        total_loss.backward()
        optimizer.step()

        running_loss += total_loss.item()

    # Evaluate accuracy at the end of each epoch
    model.eval()
    predictions = []
    targets_list = []  # Collect targets for comparison
    for images, targets in valid_loader:
        images = [img.to(device) for img in images]
        targets_list.extend(targets)  # Collect targets for accuracy calculation
        with torch.no_grad():
            pred = model(images)
        predictions.extend(pred)

    accuracy = calculate_accuracy(predictions, targets_list)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}, Accuracy: {accuracy:.4f}")


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Epoch [1/50], Loss: 29.8369, Accuracy: 0.0073
Epoch [2/50], Loss: 15.4224, Accuracy: 0.0023
Epoch [3/50], Loss: 13.7822, Accuracy: 0.0037
Epoch [4/50], Loss: 13.4805, Accuracy: 0.0053
Epoch [5/50], Loss: 12.0782, Accuracy: 0.0075
Epoch [6/50], Loss: 12.5709, Accuracy: 0.0038
Epoch [7/50], Loss: 12.7685, Accuracy: 0.0025
Epoch [8/50], Loss: 11.7870, Accuracy: 0.0021
Epoch [9/50], Loss: 12.9900, Accuracy: 0.0021
Epoch [10/50], Loss: 10.8886, Accuracy: 0.0018
Epoch [11/50], Loss: 12.1743, Accuracy: 0.0017
Epoch [12/50], Loss: 11.8598, Accuracy: 0.0039
Epoch [13/50], Loss: 11.8844, Accuracy: 0.0046
Epoch [14/50], Loss: 11.3878, Accuracy: 0.0049
Epoch [15/50], Loss: 11.9371, Accuracy: 0.0045
Epoch [16/50], Loss: 11.8020, Accuracy: 0.0035
Epoch [17/50], Loss: 11.5981, Accuracy: 0.0041
Epoch [18/50], Loss: 10.8004, Accuracy: 0.00

In [ ]:
# Evaluation phase
model.eval()
predictions = []
targets_list = []  # Collect targets for comparison

with torch.no_grad():
    for images, targets in valid_loader:
        images = [img.to(device) for img in images]
        targets_list.extend(targets)  # Collect targets for accuracy calculation
        pred = model(images)
        predictions.extend(pred)

# Ensure predictions and targets_list have the same length
print(f"Number of predictions: {len(predictions)}, Number of targets: {len(targets_list)}")

# Filter out empty predictions and collect corresponding targets
filtered_predictions = []
filtered_targets = []  # Collect corresponding targets for valid predictions
for prediction, target in zip(predictions, targets_list):
    if prediction is not None and len(prediction['boxes']) > 0:
        filtered_predictions.append(prediction)
        filtered_targets.append(target)

# Ensure filtered predictions and targets have the same length
print(f"Filtered number of predictions: {len(filtered_predictions)}, Filtered number of targets: {len(filtered_targets)}")

# Now calculate metrics
accuracy, precision, recall, f1 = calculate_metrics(filtered_predictions, filtered_targets)
print(f"Evaluation Results: "
      f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")
